<a href="https://colab.research.google.com/github/SemenutaEgor/Extreme-events-in-phenomenological-neural-networks-with-different-types-of-connections/blob/master/%D0%92%D1%80%D0%B5%D0%BC%D0%B5%D0%BD%D0%BD%D1%8B%D0%B5_%D1%80%D1%8F%D0%B4%D1%8B_%D0%B4%D0%BB%D1%8F_%D1%80%D0%B0%D0%B7%D0%BD%D1%8B%D1%85_k.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install --upgrade gupload

     |████████████████████████████████| 61kB 7.8MB/s 
     |████████████████████████████████| 81kB 9.1MB/s 
ERROR: earthengine-api 0.1.269 has requirement google-api-python-client<2,>=1.12.1, but you'll have google-api-python-client 1.7.10 which is incompatible.
  Found existing installation: google-api-python-client 1.12.8
    Uninstalling google-api-python-client-1.12.8:
      Successfully uninstalled google-api-python-client-1.12.8
  Found existing installation: click 7.1.2
    Uninstalling click-7.1.2:
      Successfully uninstalled click-7.1.2


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
import numpy as np
import scipy as scp
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp
from scipy.signal import argrelextrema

In [ ]:
#задаём параметры, функцию гамма и саму систему из двух связных нейронов
a = 1
b = 3
c = 1
d = 5
x_r = -1.6
r = 0.01
s = 5
I = 4
v_s = 2
lambd = 10
Theta = -0.25

Gamma = lambda x: 1 / (1 + np.exp(-lambd * (x - Theta)))

def two_HRs(t, X, k1, k2, k):
    x1, y1, z1, x2, y2, z2  = X
    dx1 = y1 - a*x1**3 + b*x1**2 -z1 + I - k1*(x1 - v_s) * Gamma(x2) + k * (x2 - x1)
    dy1 = c - d*x1**2 - y1
    dz1 = r*(s*(x1 - x_r) - z1)
    
    dx2 = y2 - a*x2**3 + b*x2**2 -z2 + I - k2*(x2 - v_s) * Gamma(x1) + k * (x1 - x2)
    dy2 = c - d*x2**2 - y2
    dz2 = r*(s*(x2 - x_r) - z2)
    return [dx1, dy1, dz1, dx2, dy2, dz2]

In [ ]:
ks = np.linspace(0.0, 1.0, 21, endpoint=True)
for i, k in enumerate(ks):
  k1 = -0.17
  k2 = -0.17
  
  initials = [-1.5, 0, 0, -2.5, 0, 0]
  
  sol = solve_ivp(two_HRs, [0, 5000], initials, rtol = 1e-11, atol = 1e-11, dense_output=True, args = (k1, k2, k))
  ts = sol.t
  x1s, y1s, z1s, x2s, y2s, z2s = sol.y
  
  ts = np.array(ts)
  x1s = np.array(x1s)
  x2s = np.array(x2s)
  
  xsum = x1s + x2s
  
  ts = ts[200000:]
  xsum = xsum[200000:]
  
  spikes_ind = argrelextrema(xsum, np.greater) 
  spikes = xsum[spikes_ind]
  
  mu = np.mean(spikes)
  sigma = np.std(spikes)
  
  Hs = mu + 6*sigma
  
  fig = plt.figure(figsize = (10, 5))
  ax = fig.add_subplot(1, 1, 1)
  ax.plot (ts, xsum)
  # ax.set_ylim(-3, 4) 
  ax.set_xlim(20000, 40000)
  plt.axhline(y = Hs, color ="red", linestyle ="--")
  ax.set_xlabel("$Time$", fontsize=30)
  ax.set_ylabel("$x_{II}$", fontsize=30)
  ax.tick_params(axis='both', which='major', labelsize=20)
  ax.tick_params(axis='both', which='minor', labelsize=20)
  plt.title("$k = $"+str(k), fontsize=30)
  plt.savefig(f"k{k}.png")

  !gupload --to '1Jkeb58x2sF4GSO9IlYoQ6mVkTUjN0Y0N' k{k}.png



In [ ]:
k1 = -0.17
k2 = -0.17
  
initials = [-1.5, 0, 0, -2.5, 0, 0]
  
sol = solve_ivp(two_HRs, [0, 5000], initials, rtol = 1e-11, atol = 1e-11, dense_output=True, args = (k1, k2, k))
ts = sol.t
x1s, y1s, z1s, x2s, y2s, z2s = sol.y

ts = np.array(ts)
x1s = np.array(x1s)
x2s = np.array(x2s)
  
xsum = x1s + x2s
  
ts = ts[200000:]
xsum = xsum[200000:]
  
spikes_ind = argrelextrema(xsum, np.greater) 
spikes = xsum[spikes_ind]
  
mu = np.mean(spikes)
sigma = np.std(spikes)
  
Hs = mu + 6*sigma
  
fig = plt.figure(figsize = (10, 5))
ax = fig.add_subplot(1, 1, 1)
ax.plot (ts, xsum)
# ax.set_ylim(-3, 4) 
ax.set_xlim(20000, 40000)
plt.axhline(y = Hs, color ="red", linestyle ="--")
ax.set_xlabel("$Time$", fontsize=30)
ax.set_ylabel("$x_{II}$", fontsize=30)
ax.tick_params(axis='both', which='major', labelsize=20)
ax.tick_params(axis='both', which='minor', labelsize=20)
plt.title("$k = $"+str(k), fontsize=30)
plt.savefig(f"k{k}.eps")

!gupload --to '1Jkeb58x2sF4GSO9IlYoQ6mVkTUjN0Y0N' k{k}.eps